In [ ]:
import random
import json
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen

import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy import displacy

In [ ]:
heroes_url = "https://dota2.gamepedia.com/Heroes"
hero_data = urlopen(heroes_url)
hero_html = hero_data.read()
hero_data.close()

page_soup = soup(hero_html, 'html.parser')

In [ ]:
hero_table = page_soup.findAll('div', {'class': 'mw-parser-output'})

In [ ]:
hero_division = page_soup.find_all('a', {})

In [ ]:
hero_list = []
for hero in hero_division[14:]:
    if hero['title'] == "Unreleased Content":
        break
    elif hero['title'] == "Agility" or hero['title'] == "Intelligence" or hero['title'] == "Stength":
        continue
    else:
        hero_list.append(hero['title'])
        #print(hero['title'])
#print(hero_list)
#print(len(hero_list))
hero_list.sort()


In [50]:
def add_hero_pattern(hero):
    pattern = [{"TEXT": hero}]
    matcher.add("HERO", None, pattern)

In [51]:
add_hero_pattern(hero_list[0])
print(matcher.get(0))

None


In [ ]:
filename = 'HeroList.csv'
f = open(filename, 'w')

for hero in hero_list:
    f.write(hero)
    f.write(',')

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Axe and Abaddon will take bottom lane against Zeus and Void Spirit")

In [ ]:
patterns = list(nlp.pipe(hero_list))

In [ ]:
matcher = Matcher(nlp.vocab)

#pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]
#matcher.add("IPHONE_PATTERN", None, pattern)

pattern = [{"TEXT": "Axe"}]
matcher.add("AXE", None, pattern)

matches = matcher(doc)

#matcher.add("HERO", None, *patterns)

In [ ]:
#print(matches.text)
print(hero_list)

In [ ]:
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

In [ ]:
displacy.render(doc, style = "ent", jupyter=True)

In [ ]:
for word in doc:
    print(word.text, word.pos_)

In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
# Create match patterns
pattern1 = [{"TEXT": "Axe"}]
pattern2 = [{"TEXT": "Void", "TEXT": "Spirit"}]
pattern3 = [{"LOWER": "strength"}]

In [ ]:
# Add match patterns
matcher.add("HERO", None, pattern1, pattern2)

In [ ]:
matcher.add("ATTRIBUTE", None, pattern3)
print(len(matcher))
#matcher.remove("IPHONE_PATTERN")

In [ ]:
# Create training data here
TEXTS = [
    'Axe and Slardar are strength heroes',
    'Hoodwink and Juggernaut are agility heroes',
    'Zeus and Void Spirit are intelligence heroes',
    'Riki is an agility hero',
    'Dragon Knight is a strength hero',
    'Storm Spirit is an intelligence hero',
    'Anti-Mage counters Skywrath Mage',
    'Zeus counters Riki',
    'Axe went bottom lane against Pudge',
    'Zeus tends to go to the middle lane'
]

In [ ]:
TRAINING_DATA = []

# Create Doc objects for each text in TEXTS
for doc in nlp.pipe(TEXTS):
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    # Get (start character, end character, label) tuples of matches
    entities = [(span.start_char, span.end_char, "HERO") for span in spans]
    # Format matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)

print(*TRAINING_DATA, sep="\n")

In [ ]:
# Test run examples here
string = 'Axe went bottom lane against Pudge'
find_string = 'Pudge'
print(string.find(find_string), string.find(find_string) + len(find_string))

In [ ]:
TRAINING_DATA_TEST = [
    (
        "Reddit partners with Patreon to help creators build communities",
        {"entities": [(0, 6, "WEBSITE"), (21, 28, "WEBSITE")]},
    ),
]

TRAINING_DATA_ENT_TEST = [
        ["Axe and Slardar are strength heroes", {"entities": [(0,4,"HERO"), (8,15,"HERO"), (20, 28, "ATTRIBUTE")]}],
        ["Hoodwink and Juggernaut are agility heroes", {"entities": [(0,9, "HERO"), (14,24,"HERO"), (29,36, "ATTRIBUTE")]}],
        ["Zeus and Void Spirit are intelligence heroes", {"entities": [(0,5,"HERO"), (9,20,"HERO"), (25,37,"ATTRIBUTE")]}],
        ["Riki is an agility hero", {"entities": [(0,4,"HERO"), (11,18,"ATTRIBUTE")]}],
        ["Dragon Knight is a strength hero", {"entities": [(0,13,"HERO"), (11,18,"ATTRIBUTE")]}],
        ["Storm Spirit is an intelligence hero", {"entities": [(0,12,"HERO"), (19,31,"ATTRIBUTE")]}],
        ["Anti-Mage counters Skywrath Mage", {"entities": [(0,9,"HERO"), (19,31,"ATTRIBUTE")]}],
        ["Zeus counters Riki", {"entities": [(0,4,"HERO"), (14,18,"HERO")]}],
        ["Axe went bottom lane against Pudge", {"entities": [(0,3,"HERO"), (29,34,"HERO")]}],
        ["Zeus tends to go to the middle lane", {"entities": [(0,4,"HERO")]}],
]

#json_string = json.dumps(TRAINING_DATA_ENT_TEST)
#print(json_string)

with open('heroentlist.json', 'w') as j:
    json.dump(TRAINING_DATA_ENT_TEST, j)


In [ ]:
TRAINING_DATA_ENT = [
    (
        "Axe and Slardar are strength heroes", {"entities": [(0,4,"HERO"), (8,15,"HERO"), (20, 28, "ATTRIBUTE")]},
        "Hoodwink and Juggernaut are agility heroes", {"entities": [(0,9, "HERO"), (14,24,"HERO"), (29,36, "ATTRIBUTE")]},
        "Zeus and Void Spirit are intelligence heroes", {"entities": [(0,5,"HERO"), (9,20,"HERO"), (25,37,"ATTRIBUTE")]},
        "Riki is an agility hero", {"entities": [(0,4,"HERO"), (11,18,"ATTRIBUTE")]},
        "Dragon Knight is a strength hero", {"entities": [(0,13,"HERO"), (11,18,"ATTRIBUTE")]},
        "Storm Spirit is an intelligence hero", {"entities": [(0,12,"HERO"), (19,31,"ATTRIBUTE")]},
        "Anti Mage counters Skywrath Mage", {"entities": [(0,9,"HERO"), (19,31,"ATTRIBUTE")]},
        "Zeus counters Riki", {"entities": [(0,4,"HERO"), (14,18,"HERO")]},
        "Axe went bottom lane against Pudge", {"entities": [(0,3,"HERO"), (29,34,"HERO")]},
        "Zeus tends to go to the middle lane", {"entities": [(0,4,"HERO")]},
    ),
]

In [ ]:
# From Chapter 4 Section 5
# Create new pipeline

with open("heroentlist.json", encoding="utf8") as h:
    TRAINING_DATA = json.loads(h.read())

# Save a model
# nlp.to_disk(disk_path)

# Example example:
# ['How to preorder the iPhone X', {'entities': [[20, 28, 'GADGET']]}]
# {"entities": [(0, 6, "WEBSITE"), (21, 28, "WEBSITE")]}

# Create blank Enlgish model
nlp = spacy.blank("en")

# Create blank entity recognizer and add it to pipeline
ner = nlp.create_pipe("ner")

nlp.add_pipe(ner)

# Add a new label
ner.add_label("HERO")
ner.add_label("ABILITY")
ner.add_label("ATTRIBUTE")

# LABELS = ["HERO", "ABILITY", "ATTRIBUTES"]

nlp.begin_training()

# train for 10 iterations
for itn in range(10):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)

print("Job's done")

In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
# Create match patterns
pattern1 = [{"TEXT": "Axe"}]
pattern2 = [{"TEXT": "Void", "TEXT": "Spirit"}]
pattern3 = [{"LOWER": "strength"}]

In [ ]:
matcher.add("HERO", None, pattern1, pattern2)
matcher.add("ATTRIBUTE", None, pattern3)

In [52]:
print(nlp.pipe_names)

['ner']


In [ ]:
doc = nlp("Anti Mage fought Axe in the top lane")

matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

#for word in doc:
    #print(word.text, word.label)